# 🛠️ Uso Avanzato degli Strumenti con i Modelli GitHub (.NET)

## 📋 Obiettivi di Apprendimento

Questo notebook dimostra i modelli di integrazione degli strumenti di livello aziendale utilizzando il Microsoft Agent Framework in .NET con i Modelli GitHub. Imparerai a costruire agenti sofisticati con strumenti multipli e specializzati, sfruttando il forte typing di C# e le funzionalità aziendali di .NET.

**Capacità Avanzate degli Strumenti che Apprenderai:**
- 🔧 **Architettura Multi-Strumento**: Creazione di agenti con capacità specializzate multiple
- 🎯 **Esecuzione di Strumenti Type-Safe**: Sfruttare la validazione a tempo di compilazione di C#
- 📊 **Modelli di Strumenti Aziendali**: Progettazione di strumenti pronti per la produzione e gestione degli errori
- 🔗 **Composizione degli Strumenti**: Combinare strumenti per flussi di lavoro aziendali complessi

## 🎯 Vantaggi dell'Architettura degli Strumenti in .NET

### Caratteristiche degli Strumenti Aziendali
- **Validazione a Tempo di Compilazione**: Il forte typing garantisce la correttezza dei parametri degli strumenti
- **Iniezione delle Dipendenze**: Integrazione del contenitore IoC per la gestione degli strumenti
- **Pattern Async/Await**: Esecuzione non bloccante degli strumenti con gestione adeguata delle risorse
- **Logging Strutturato**: Integrazione del logging per il monitoraggio dell'esecuzione degli strumenti

### Modelli Pronti per la Produzione
- **Gestione delle Eccezioni**: Gestione completa degli errori con eccezioni tipizzate
- **Gestione delle Risorse**: Pattern di smaltimento adeguato e gestione della memoria
- **Monitoraggio delle Prestazioni**: Metriche integrate e contatori di prestazioni
- **Gestione della Configurazione**: Configurazione type-safe con validazione

## 🔧 Architettura Tecnica

### Componenti Principali degli Strumenti in .NET
- **Microsoft.Extensions.AI**: Livello di astrazione unificato per gli strumenti
- **Microsoft.Agents.AI**: Orchestrazione degli strumenti di livello aziendale
- **Integrazione con i Modelli GitHub**: Client API ad alte prestazioni con pooling delle connessioni

### Pipeline di Esecuzione degli Strumenti
```csharp
User Request → Agent Analysis → Tool Selection → Type Validation
                 ↓               ↓              ↓
         Parameter Binding → Tool Execution → Result Processing → Response
```

## 🛠️ Categorie e Modelli degli Strumenti

### 1. **Strumenti di Elaborazione Dati**
- **Validazione degli Input**: Forte typing con annotazioni dei dati
- **Operazioni di Trasformazione**: Conversione e formattazione dei dati type-safe
- **Logica Aziendale**: Strumenti di calcolo e analisi specifici del dominio
- **Formattazione degli Output**: Generazione di risposte strutturate

### 2. **Strumenti di Integrazione**
- **Connettori API**: Integrazione di servizi RESTful con HttpClient
- **Strumenti per Database**: Integrazione di Entity Framework per l'accesso ai dati
- **Operazioni sui File**: Operazioni sicure sul file system con validazione
- **Servizi Esterni**: Modelli di integrazione con servizi di terze parti

### 3. **Strumenti Utilitari**
- **Elaborazione Testuale**: Utilità per la manipolazione e formattazione delle stringhe
- **Operazioni su Date/Ora**: Calcoli su date/ora sensibili alla cultura
- **Strumenti Matematici**: Calcoli di precisione e operazioni statistiche
- **Strumenti di Validazione**: Validazione delle regole aziendali e verifica dei dati

## ⚙️ Prerequisiti e Configurazione

**Ambiente di Sviluppo:**
- SDK .NET 9.0 o superiore
- Visual Studio 2022 o VS Code con estensione C#
- Accesso all'API dei Modelli GitHub

**Pacchetti NuGet Necessari:**
```xml
<PackageReference Include="Microsoft.Extensions.AI" Version="9.9.0" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
```

**Configurazione dell'Ambiente (.env file):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

Pronto a costruire agenti di livello aziendale con potenti capacità di strumenti type-safe in .NET? Progettiamo soluzioni professionali! 🏢⚡


In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.1"

Installed Packages Microsoft.Extensions.AI, 9.9.1

In [ ]:
#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.2

In [ ]:
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.251001.3"

Installed Packages microsoft.agents.ai, 1.0.0-preview.251001.2

In [4]:
// #r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

In [5]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [6]:
using System;
using System.ComponentModel;
using System.ClientModel;

using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using OpenAI;

In [7]:
 using DotNetEnv;

In [8]:
Env.Load("../../../.env");

In [9]:
[Description("Provides a random vacation destination.")]
static string GetRandomDestination()
{
    var destinations = new List<string>
    {
        "Paris, France",
        "Tokyo, Japan",
        "New York City, USA",
        "Sydney, Australia",
        "Rome, Italy",
        "Barcelona, Spain",
        "Cape Town, South Africa",
        "Rio de Janeiro, Brazil",
        "Bangkok, Thailand",
        "Vancouver, Canada"
    };

    var random = new Random();
    int index = random.Next(destinations.Count);
    return destinations[index];
}

In [10]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [11]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [12]:

var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [13]:
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    instructions:"You are a helpful AI Agent that can help plan vacations for customers at random destinations", tools: [AIFunctionFactory.Create((Func<string>)GetRandomDestination)]);

In [14]:
AgentThread thread = agent.GetNewThread();

In [15]:
Console.WriteLine(await agent.RunAsync("Plan me a day trip",thread));

Your day trip will be in Paris, France! Here's a proposed itinerary for your day:

### Morning
- **Breakfast at a Local Café**: Start your day with a traditional French breakfast. Try a croissant and café au lait at a local café.
- **Visit the Eiffel Tower**: Arrive early to beat the crowds. Enjoy the stunning views of the city from the top.

### Late Morning
- **Stroll Along the Seine**: Take a leisurely walk along the Seine River and enjoy the picturesque views.
- **Visit Notre-Dame Cathedral**: Explore this iconic Gothic cathedral and its stunning architecture.

### Lunch
- **Lunch at a Bistro**: Treat yourself to a classic French lunch at a nearby bistro. Consider trying coq au vin or a delicious quiche.

### Afternoon
- **Explore the Louvre**: Spend your afternoon at the Louvre Museum. While you may not see everything, be sure to check out the Mona Lisa and other famous artworks.
- **Walk Through the Tuileries Garden**: After the museum, relax in the beautiful Tuileries Garden.

#

In [16]:
Console.WriteLine(await agent.RunAsync("I don't like that destination. Plan me another vacation.",thread));

Your new vacation destination is Rio de Janeiro, Brazil! Here's a proposed itinerary for your trip:

### Day 1: Arrival in Rio
- **Check-in**: Arrive and settle into your accommodation.
- **Evening at Copacabana Beach**: Stroll along the famous Copacabana Beach. Enjoy the vibrant atmosphere and watch the sunset.

### Day 2: Exploring the City
- **Visit Christ the Redeemer**: Start your day with a visit to the iconic Christ the Redeemer statue. Take the train up to the top for breathtaking views.
- **Explore Santa Teresa**: Wander through the charming streets of Santa Teresa, known for its colonial-style houses, art studios, and local shops.
- **Lunch at a Local Restaurant**: Enjoy traditional Brazilian cuisine, such as feijoada (black bean stew).
- **Afternoon at Sugarloaf Mountain**: Take a cable car ride to Sugarloaf Mountain for panoramic views of the city and coastline.

### Day 3: Cultural Experience
- **Visit the Selarón Steps**: Walk up the colorful Selarón Steps, an iconic mosa


---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione AI [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
